In [1]:
import pandas as pd
import numpy as np
import pickle
import random
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
df = pd.read_csv('../scraping/all_data_after_race1.csv').drop(columns = ['Unnamed: 0', 'qualifying_time', 'driver'])

In [3]:
df = df.dropna()

In [4]:
df.reset_index(inplace=True, drop=True)

In [5]:
test_df = pd.DataFrame()
train_df = pd.DataFrame()

for s in np.unique(df['season']):
    for r in np.unique(df['round']):        
        rand = random.random()
        if rand<.2:
            test_df = pd.concat([test_df, df[(df['season']==s) & (df['round']==r)]])
        else: 
            train_df = pd.concat([train_df, df[(df['season']==s) & (df['round']==r)]])

    X_train = train_df.drop(columns=["podium"])
    X_test = test_df.drop(columns=["podium"])
y_train = train_df['podium']
y_test = test_df['podium']

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)

LinearRegression()

In [7]:
predictions = model.predict(X_test)

In [8]:
results = test_df.copy()

In [9]:
results['predictions'] = predictions

In [10]:
score = 0
preds = pd.DataFrame()
counter = 0
for year in results.season.unique():
    for circuit in results['round'].unique():
        test = results[(results.season == year) & (results['round'] == circuit)]
        if test.shape[0]>0:
            counter += 1
#             X_test = test.drop(['podium'], axis=1)
            y_test = test.podium


            # make predictions
            prediction_df = pd.DataFrame(
                test['predictions'])
            prediction_df = prediction_df.rename(columns={'predictions':'results'})
            prediction_df['podium'] = y_test#.reset_index(drop=True)
            prediction_df['actual'] = prediction_df.podium.map(
                lambda x: 1 if x == 1 else 0)
            prediction_df.sort_values('results', ascending=True, inplace=True)
            prediction_df.reset_index(inplace=True, drop=True)
            prediction_df['predicted'] = prediction_df.index
            prediction_df['predicted'] = prediction_df.predicted.map(
                lambda x: 1 if x == 0 else 0)
            prediction_df['round'] = prediction_df.predicted.map(
                lambda x: circuit)
            preds = preds.append(prediction_df)

            score += precision_score(prediction_df.actual,
                                     prediction_df.predicted)

model_score = score / counter

/var/folders/gr/0kfd3y813t701km43_czv8_c0000gn/T/ipykernel_51227/2864874956.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds = preds.append(prediction_df)
/var/folders/gr/0kfd3y813t701km43_czv8_c0000gn/T/ipykernel_51227/2864874956.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds = preds.append(prediction_df)
/var/folders/gr/0kfd3y813t701km43_czv8_c0000gn/T/ipykernel_51227/2864874956.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds = preds.append(prediction_df)
/var/folders/gr/0kfd3y813t701km43_czv8_c0000gn/T/ipykernel_51227/2864874956.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds = preds.append

/var/folders/gr/0kfd3y813t701km43_czv8_c0000gn/T/ipykernel_51227/2864874956.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds = preds.append(prediction_df)
/var/folders/gr/0kfd3y813t701km43_czv8_c0000gn/T/ipykernel_51227/2864874956.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds = preds.append(prediction_df)
/var/folders/gr/0kfd3y813t701km43_czv8_c0000gn/T/ipykernel_51227/2864874956.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds = preds.append(prediction_df)
/var/folders/gr/0kfd3y813t701km43_czv8_c0000gn/T/ipykernel_51227/2864874956.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds = preds.append

In [11]:
model_score

0.44366197183098594

In [12]:
pred_df = pd.read_csv('pred_race_2.csv')

In [13]:
pred_df = pred_df.sort_values(by='grid')

In [20]:
pred_df

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,driver_points,...,circuit_id_fuji,circuit_id_pacific,circuit_id_europe,circuit_id_fair_park,circuit_id_dijon_prenois,circuit_id_rio,direction_a,direction_c,direction_nan,prediction
0,2023,2,True,False,True,False,False,perez,1,18,...,0,0,0,0,0,0,1,0,0,1.057420e+13
2,2023,2,True,False,True,False,False,alonso,2,15,...,0,0,0,0,0,0,1,0,0,1.057420e+13
3,2023,2,True,False,True,False,False,russell,3,6,...,0,0,0,0,0,0,1,0,0,1.057420e+13
4,2023,2,True,False,True,False,False,sainz,4,12,...,0,0,0,0,0,0,1,0,0,1.057420e+13
5,2023,2,True,False,True,False,False,stroll,5,8,...,0,0,0,0,0,0,1,0,0,1.057420e+13
6,2023,2,True,False,True,False,False,ocon,6,0,...,0,0,0,0,0,0,1,0,0,1.057420e+13
7,2023,2,True,False,True,False,False,hamilton,7,10,...,0,0,0,0,0,0,1,0,0,1.057420e+13
8,2023,2,True,False,True,False,False,piastri,8,0,...,0,0,0,0,0,0,1,0,0,1.057420e+13
9,2023,2,True,False,True,False,False,gasly,9,2,...,0,0,0,0,0,0,1,0,0,1.057420e+13
10,2023,2,True,False,True,False,False,hulkenberg,10,0,...,0,0,0,0,0,0,1,0,0,1.057420e+13


In [15]:
pred_df2 = pred_df.drop(columns=['driver'])

In [16]:
pred_df["prediction"] = model.predict(pred_df2)
pred_df2["prediction"] = model.predict(pred_df2)

/Users/salma/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/Users/salma/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [25]:
pred_df.head()

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,driver_points,...,circuit_id_fuji,circuit_id_pacific,circuit_id_europe,circuit_id_fair_park,circuit_id_dijon_prenois,circuit_id_rio,direction_a,direction_c,direction_nan,prediction
0,2023,2,True,False,True,False,False,perez,1,18,...,0,0,0,0,0,0,1,0,0,1.057420e+13
2,2023,2,True,False,True,False,False,alonso,2,15,...,0,0,0,0,0,0,1,0,0,1.057420e+13
3,2023,2,True,False,True,False,False,russell,3,6,...,0,0,0,0,0,0,1,0,0,1.057420e+13
4,2023,2,True,False,True,False,False,sainz,4,12,...,0,0,0,0,0,0,1,0,0,1.057420e+13
5,2023,2,True,False,True,False,False,stroll,5,8,...,0,0,0,0,0,0,1,0,0,1.057420e+13


In [22]:
pred_df.sort_values(by=['prediction', 'driver'])[['driver']]

,driver
0,perez
2,alonso
4,sainz
3,russell
6,ocon
5,stroll
8,piastri
7,hamilton
1,leclerc
14,max_verstappen


In [18]:
with open("linear_race2.pickle", 'wb') as f:
        pickle.dump(model, f)

In [13]:
#perez then alonso